In [1]:
from collections import defaultdict
from collections import Counter
import json
import math
import string
import time
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import wordnet
#nltk.download('wordnet')
import spacy
nlp = spacy.load('en_core_web_lg')

from nltk.tokenize import TreebankWordTokenizer

In [2]:
data = pd.read_csv("fun_fact_title.csv")

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,8,10,11,13,14,15,16,17,19,20,21,22,23,24,25,26,28,29,30,31,33,37,38,43,44,47,48,49,50,51,52,54,55,57,58,60,61,62,64,65,71,75,87,89,91,92,93,94,97,101,104,105,106,107,110,111,112,113,114,115,118,119,120,121,122,129,133,135,137,138,139,140,142,143,148,152,155,174,176,178,179,180,181,187,191,192,193,194,197,199,201,202,203,209) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
documents = data["title"].tolist()
score = data ["score"].tolist()

In [4]:
doc_dict = {}
for i in range(len(documents)):
    doc_dict[i] = documents[i]

In [5]:
#build inverted index
inverted_idx = defaultdict(list)
for d in range(len(documents)):
    words = documents[d].split()
    #for w in words:
        #inverted_idx[w.lower()] += 1
    c = Counter(words)
    for w in c:
        #inverted_idx[w.lower()].append((d, c[w]))
        inverted_idx[w.lower()].append(d)

In [7]:
vocab = [w for w in nlp.vocab if w.is_lower and w.prob >= -15 and w.vector.any()]
def most_similar(word, top=10):
    by_similarity = sorted(vocab, key=lambda w: word.similarity(w), reverse=True)
    return by_similarity[:top]

In [8]:
#query = "apple"
#synonyms = set(most_similar(query))
def booleanSearchWithSyn(query):
    queries = query.split()
    doc_num = set()
    for word in queries:
        synonyms = set([w.lower_ for w in most_similar(nlp.vocab[word])])
        synonyms.remove(word)
        syn_words = set()
        for syn in synonyms:
            x = set(inverted_idx[syn]) #set of integers  
            syn_words = syn_words.union(x)
        if len(doc_num) == 0:
            doc_num = syn_words
        else:
            doc_num = doc_num.intersection(syn_words)
    return doc_num
#doc_num_syn = booleanSearchWithSyn(query)

In [9]:
def booleanSearch(query):
    queries = query.split()
    doc_num = set()
    for word in queries:
        x = set(inverted_idx[word]) #set of integers  
        if len(doc_num) == 0:
            doc_num = x
        else:
            doc_num = doc_num.intersection(x)
    return doc_num
#doc_num = booleanSearch(query)

In [25]:
def rankResults(doc_num):
    relevant = []
    for d in doc_num:
        relevant.append((d, score[d]))
    relevant.sort(key=lambda x: x[1], reverse = True)
    return relevant

def search(query, top = 10):
    query = query.lower()
    doc_num = booleanSearch(query)
    rel = rankResults(doc_num)[:top]
    if len(doc_num) < top:
        doc_num_syn = booleanSearchWithSyn(query)
        rel_syn = rankResults(doc_num_syn)
        for i in rel_syn:
            if len(rel) < top:
                if i not in rel:
                    rel.append(i)
            else:
                break
    results = []
    for r,_ in rel:
        results.append(doc_dict[r])
        #print (doc_dict[r])

search("basketball space")

0
SYN
Fun Fact: Hidden inside Grand Central Station, in New York, is the Vanderbilt Tennis Club.
Fun fact I just learned: EVERY baseball used in MLB (and the minors) MUST be rubbed down with mud from a specific area of the New Jersey side of the Delaware River...
Fun fact I just learned: EVERY baseball used in MLB (and the minors) MUST be first rubbed down with mud from a specific area of the New Jersey side of the Delaware River...
3
